# 垂直联邦XGB (MPC XGBoost)

> 以下代码仅供演示。出于系统安全考虑，请 **不要** 直接用于生产。


In [61]:
from sklearn.metrics import roc_auc_score

import secretflow as sf
from secretflow.data import FedNdarray, PartitionWay
from secretflow.device.driver import reveal
from secretflow.ml.boost.ss_xgb_v import Xgb
import pprint

pp = pprint.PrettyPrinter(depth=4)

# Check the version of your SecretFlow
print('The version of SecretFlow: {}'.format(sf.__version__))

The version of SecretFlow: 1.7.0b0


In [62]:
# process_utils.py
import psutil
import os
import platform


def find_process_by_port(port):
    for proc in psutil.process_iter(["pid", "name"]):
        try:
            connections = proc.connections(kind="inet")
            for conn in connections:
                if conn.laddr.port == port:
                    return proc.pid
        except (psutil.AccessDenied, psutil.NoSuchProcess):
            continue
    return None


def kill_process(pid):
    try:
        if platform.system() == "Windows":
            os.system(f"taskkill /PID {pid} /F")
        else:
            os.system(f"kill -9 {pid}")
        print(f"Process with PID {pid} has been terminated.")
    except Exception as e:
        print(f"Error terminating process with PID {pid}: {e}")


def kill_port_process(port: int):
    pid = find_process_by_port(port)

    if pid:
        print(f"Process found using port {port}: PID {pid}")
        kill_process(pid)
    else:
        print(f"No process found using port {port}")


pyu_port = 16307
spu_port = 11666

kill_port_process(port=pyu_port)
kill_port_process(port=spu_port)

import secretflow as sf

# Check the version of your SecretFlow
print("The version of SecretFlow: {}".format(sf.__version__))

# In case you have a running secretflow runtime already.
sf.shutdown()

pyu_port = 16307
spu_port = 11666


cluster_config = {
    "parties": {
        "alice": {
            # replace with alice's real address.
            "address": "ecm-01:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
        "bob": {
            # replace with bob's real address.
            "address": "ecm-02:" + str(pyu_port),
            "listen_addr": "0.0.0.0:" + str(pyu_port),
        },
    },
    "self_party": "alice",
}

tls_config = {
    "ca_cert": "/home/beng003/certificate/bob_ca.crt",
    "cert": "/home/beng003/certificate/alice_server_cert.crt",
    "key": "/home/beng003/certificate/alice_server_key.key",
}


sf.init(address="ecm-01:6379", cluster_config=cluster_config, tls_config=tls_config)
alice = sf.PYU("alice")
bob = sf.PYU("bob")
print("Alice and Bob are ready to go!")

import spu

cluster_def = {
    "nodes": [
        {
            "party": "alice",
            # The address for other peers.
            "address": "ecm-01:" + str(spu_port),
            # The listen address of this node.
            # Optional. Address will be used if listen_address is empty.
            "listen_address": "0.0.0.0:" + str(spu_port),
        },
        {
            "party": "bob",
            "address": "ecm-02:" + str(spu_port),
            "listen_address": "0.0.0.0:" + str(spu_port),
        },
    ],
    "runtime_config": {
        "protocol": spu.spu_pb2.SEMI2K,
        "field": spu.spu_pb2.FM128,
        "sigmoid_mode": spu.spu_pb2.RuntimeConfig.SIGMOID_REAL,
    },
}

link_desc = {
    "recv_timeout_ms": 6000,
}

spu_device = sf.SPU(cluster_def=cluster_def, link_desc=link_desc)
print("*****************************************************Alice SPU")

/tmp/ipykernel_923696/1211501630.py:10: DeprecationWarning: connections() is deprecated and will be removed; use net_connections() instead
  connections = proc.connections(kind="inet")


No process found using port 16307
No process found using port 11666
The version of SecretFlow: 1.7.0b0


2024-08-03 16:37:49,664	INFO worker.py:1540 -- Connecting to existing Ray cluster at address: ecm-01:6379...
2024-08-03 16:37:49,681	INFO worker.py:1724 -- Connected to Ray cluster.
2024-08-03 16:37:49.721 INFO api.py:233 [alice] -- [Anonymous_job] Started rayfed with {'CLUSTER_ADDRESSES': {'alice': '0.0.0.0:16307', 'bob': 'ecm-02:16307'}, 'CURRENT_PARTY_NAME': 'alice', 'TLS_CONFIG': {'ca_cert': '/home/beng003/certificate/bob_ca.crt', 'cert': '/home/beng003/certificate/alice_server_cert.crt', 'key': '/home/beng003/certificate/alice_server_key.key'}}
2024-08-03 16:37:51.146 INFO barriers.py:284 [alice] -- [Anonymous_job] Succeeded to create receiver proxy actor.
(ReceiverProxyActor pid=938319) 2024-08-03 16:37:51.140 INFO grpc_proxy.py:359 [alice] -- [Anonymous_job] ReceiverProxy binding port 16307, options: (('grpc.enable_retries', 1), ('grpc.so_reuseport', 0), ('grpc.max_send_message_length', 524288000), ('grpc.max_receive_message_length', 524288000), ('grpc.service_config', '{"method

Alice and Bob are ready to go!
*****************************************************Alice SPU


(raylet) [2024-08-03 17:29:21,153 E 16009 16009] (raylet) node_manager.cc:3024: 1 Workers (tasks / actors) killed due to memory pressure (OOM), 0 Workers crashed due to other reasons at node (ID: 47b2c7b94af672612aced3c2e8a79c833f7f9db594029492d2e7085e, IP: ecm-01) over the last time period. To see more information about the Workers killed on this node, use `ray logs raylet.out -ip ecm-01`
(raylet) 
(raylet) Refer to the documentation on how to address the out of memory issue: https://docs.ray.io/en/latest/ray-core/scheduling/ray-oom-prevention.html. Consider provisioning more memory on this node or reducing task parallelism by requesting more CPUs per task. To adjust the kill threshold, set the environment variable `RAY_memory_usage_threshold` when starting Ray. To disable worker killing, set the environment variable `RAY_memory_monitor_refresh_ms` to zero.
(raylet) [2024-08-03 17:31:21,156 E 16009 16009] (raylet) node_manager.cc:3024: 1 Workers (tasks / actors) killed due to memory p

## 数据准备

我们将准备一个垂直数据集。

In [63]:
from sklearn.datasets import load_breast_cancer

ds = load_breast_cancer()
x, y = ds['data'], ds['target']

v_data = FedNdarray(
    {
        alice: (alice(lambda: x[:, :15])()),
        bob: (bob(lambda: x[:, 15:])()),
    },
    partition_way=PartitionWay.VERTICAL,
)
label_data = FedNdarray(
    {alice: (alice(lambda: y)())},
    partition_way=PartitionWay.VERTICAL,
)

In [13]:
import pandas as pd

df = pd.DataFrame(ds['data'], columns=ds["feature_names"])
df

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,25.380,17.33,184.60,2019.0,0.16220,0.66560,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,24.990,23.41,158.80,1956.0,0.12380,0.18660,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,23.570,25.53,152.50,1709.0,0.14440,0.42450,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,14.910,26.50,98.87,567.7,0.20980,0.86630,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,22.540,16.67,152.20,1575.0,0.13740,0.20500,0.4000,0.1625,0.2364,0.07678
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
564,21.56,22.39,142.00,1479.0,0.11100,0.11590,0.24390,0.13890,0.1726,0.05623,...,25.450,26.40,166.10,2027.0,0.14100,0.21130,0.4107,0.2216,0.2060,0.07115
565,20.13,28.25,131.20,1261.0,0.09780,0.10340,0.14400,0.09791,0.1752,0.05533,...,23.690,38.25,155.00,1731.0,0.11660,0.19220,0.3215,0.1628,0.2572,0.06637
566,16.60,28.08,108.30,858.1,0.08455,0.10230,0.09251,0.05302,0.1590,0.05648,...,18.980,34.12,126.70,1124.0,0.11390,0.30940,0.3403,0.1418,0.2218,0.07820
567,20.60,29.33,140.10,1265.0,0.11780,0.27700,0.35140,0.15200,0.2397,0.07016,...,25.740,39.42,184.60,1821.0,0.16500,0.86810,0.9387,0.2650,0.4087,0.12400


### 参数准备

In [64]:
params = {
    "num_boost_round": 10,  # Number of boosting iterations, default is 10
    "max_depth": 5,  # Maximum depth of a tree, default is 5
    "learning_rate": 0.3,  # Step size shrinkage, default is 0.3
    "objective": "logistic",  # Learning objective, default is 'logistic'
    "reg_lambda": 0.1,  # L2 regularization term, default is 0.1
    "subsample": 1,  # Subsample ratio of the training instances, default is 1
    "colsample_by_tree": 1,  # Subsample ratio of columns, default is 1
    "sketch_eps": 0.1,  # Number of bins for sketching, default is 0.1
    "base_score": 0,  # Initial prediction score, default is 0
    "seed": 42,  # Pseudorandom number generator seed, default is 42
}
pp.pprint(params)

{'base_score': 0,
 'colsample_by_tree': 1,
 'learning_rate': 0.3,
 'max_depth': 5,
 'num_boost_round': 10,
 'objective': 'logistic',
 'reg_lambda': 0.1,
 'seed': 42,
 'sketch_eps': 0.1,
 'subsample': 1}


## 运行 Xgb

我们使用 spu 设备创建一个 Xgb 对象，并拟合数据。

In [65]:
xgb = Xgb(spu_device)
model = xgb.train(params, v_data, label_data)

2024-08-03 16:38:25.315 INFO proxy.py:180 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party alice.
2024-08-03 16:38:25.355 INFO proxy.py:180 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party bob.
2024-08-03 16:38:25.357 INFO booster.py:167 [alice] -- [Anonymous_job] fragment_count 1
2024-08-03 16:38:27.011 INFO booster.py:185 [alice] -- [Anonymous_job] prepare time 2.946911334991455s
2024-08-03 16:38:27.188 INFO booster.py:198 [alice] -- [Anonymous_job] global_setup time 0.17596864700317383s
2024-08-03 16:38:33.430 INFO booster.py:217 [alice] -- [Anonymous_job] build & infeed bucket_map fragments [0, 0]
2024-08-03 16:38:33.431 INFO booster.py:220 [alice] -- [Anonymous_job] build & infeed bucket_map time 6.2414374351501465s
2024-08-03 16:38:33.569 INFO booster.py:233 [alice] -- [Anonymous_job] init_pred time 0.13740897178649902s
2024-0

(_run pid=938515) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
(_run pid=938515) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=938515) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory


(_run pid=938515) [2024-08-03 16:38:27.228] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7
(SPURuntime(device_id=None, party=alice) pid=938424) 2024-08-03 16:38:33.670 [info] [thread_pool.cc:ThreadPool:30] Create a fixed thread pool with size 7


(_run pid=938516) INFO:jax._src.xla_bridge:Unable to initialize backend 'cuda': 
(_run pid=938516) INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': module 'jaxlib.xla_extension' has no attribute 'GpuAllocatorConfig'
(_run pid=938516) INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': INTERNAL: Failed to open libtpu.so: libtpu.so: cannot open shared object file: No such file or directory
(_run pid=938516) INFO:jax._src.xla_bridge:Unable to initialize backend 'interpreter': jaxlib.xla_extension.Client: no constructor defined!


(XgbTreeWorker pid=938892) [2024-08-03 16:40:29.322] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7
(XgbTreeWorker pid=946529) [2024-08-03 16:59:47.761] [info] [thread_pool.cc:30] Create a fixed thread pool with size 7


## 模型评估

现在我们可以将模型输出与真实标签进行比较。

In [66]:
yhat = model.predict(v_data)
yhat = reveal(yhat)
print(f"auc: {roc_auc_score(y, yhat)}")

2024-08-03 16:59:45.982 INFO proxy.py:180 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party alice.
2024-08-03 16:59:46.021 INFO proxy.py:180 [alice] -- [Anonymous_job] Create proxy actor <class 'secretflow.ml.boost.ss_xgb_v.core.tree_worker.XgbTreeWorker'> with party bob.


auc: 1.0


## 模型权重

现在我们可以将模型输出与真实标签进行比较。

In [69]:
a=sf.reveal(model.get_weights())
a

[array([[ 0.591123  ],
        [ 0.3934378 ],
        [ 0.        ],
        [-0.42856592],
        [ 0.5492958 ],
        [-0.2727273 ],
        [ 0.27272728],
        [-0.44680843],
        [ 0.5555502 ],
        [ 0.        ],
        [ 0.        ],
        [-0.42856592],
        [ 0.42856592],
        [ 0.        ],
        [ 0.        ],
        [-0.58973515],
        [ 0.54545456],
        [ 0.        ],
        [ 0.        ],
        [-0.42856592],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [-0.5624961 ],
        [ 0.        ],
        [-0.5985111 ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ],
        [ 0.        ]], dtype=float32),
 array([[ 0.3981441 ],
        [ 0.        ],
        [-0.10535111],
        [ 0.37898457],
        [ 0.2834919 ],
        [-0.5448195 ],
        [-0.76084554],
        [-0.38050643],
        [ 0.36560106],
        [ 0.        ],
        [-0.29212

In [71]:
import numpy as np

pd.DataFrame(np.array(a).reshape(-1, 32))

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.591123,0.393438,0.000000,-0.428566,0.549296,-0.272727,0.272727,-0.446808,0.555550,0.000000,...,0.000000,-0.562496,0.000000,-0.598511,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.398144,0.000000,-0.105351,0.378985,0.283492,-0.544819,-0.760846,-0.380506,0.365601,0.000000,...,0.267104,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.395093
2,0.267190,0.350040,0.298861,-0.370346,0.000000,0.209736,0.000000,-0.326099,0.266274,0.000000,...,-0.516012,0.000000,0.367871,0.000000,-0.320124,0.000000,-0.278211,0.306341,-0.862369,-0.356445
3,0.333209,0.000000,0.334066,-0.039387,-0.386179,0.000000,0.299116,0.000000,0.272315,0.000000,...,0.000000,0.000000,0.321843,0.000000,-0.294073,0.000000,0.247655,-0.301722,0.000000,-0.340614
4,0.322361,0.000000,0.318057,-0.115916,0.000000,-0.295220,0.245763,0.000000,0.000000,0.195606,...,0.000000,0.221430,0.000000,-0.238600,0.000000,0.281949,0.278310,-0.269318,0.129511,-0.325354
5,0.315007,0.000000,0.307965,-0.034883,-0.280557,0.115076,0.274806,-0.179849,-0.032153,0.248487,...,0.000000,-0.242211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.313087,0.000000
6,0.310305,0.081967,0.000000,-0.145438,0.305083,0.000000,0.098242,-1.073994,0.283664,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.308136,0.000000,0.000000
7,0.308176,0.000000,-0.228805,0.215403,0.287332,0.000000,-0.430497,0.080396,0.201374,0.000000,...,0.194979,-0.199389,0.000000,-0.182024,0.000000,0.204898,0.171272,-0.127115,-0.301968,0.000000
8,0.304036,0.000000,-0.241433,0.219387,0.076250,-0.237975,0.242425,-0.134733,0.207913,-0.078375,...,0.000000,-0.203866,0.000000,-0.301327,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.302591,0.000664,-0.183873,0.188923,0.274583,0.000000,-0.362964,0.182483,0.233087,0.000000,...,0.263485,0.000000,0.200773,0.000000,-0.203758,0.000000,0.000000,0.113404,-0.073246,-0.300710
